# The Neuron and The Perceptron

## 🎯 Learning Objectives

By the end of this tutorial, you will be able to:

- Implement an artificial neuron (perceptron) and understand its computational properties
- Implement a biologically-inspired neuron (Leaky Integrate-and-Fire) with membrane dynamics
- Model synaptic input using conductance-based synapses
- Implement short-term plasticity (synaptic depression) with finite vesicle pools
- Compare input, processing, and output between artificial and biological neurons

## 📚 Prerequisites

- **Conceptual**: Complete Tutorial 01 (What is NeuroAI?) for background
- **Technical**: Basic Python programming, familiarity with NumPy


---

## Setup and Imports


In [ ]:
import doctest

import numpy as np

from neuroai.plotting import (
    plot_activation_functions_comparison,
    plot_complete_neuron,
    plot_fi_curve,
    plot_io_comparison,
    plot_layer_response,
    plot_lif_response,
    plot_perceptron_computation,
    plot_stp_comparison,
    plot_synapse_comparison,
    plot_synaptic_response,
)

# Set random seed for reproducibility
rng = np.random.default_rng(42)

---

## Part 1: The Perceptron (Artificial Neuron)

The perceptron, introduced by Frank Rosenblatt in 1958, is the simplest artificial neuron. Despite its simplicity, it forms the building block of modern deep learning.

### The Perceptron Computation

1. **Input**: Receive a vector of inputs $\mathbf{x} = [x_1, x_2, ..., x_n]$
2. **Processing**:
   - Compute weighted sum: $z = \sum_{i=1}^{n} w_i x_i + b = \mathbf{w} \cdot \mathbf{x} + b$
   - Apply activation function: $y = \sigma(z)$
3. **Output**: A single scalar value $y$


### 1.1 Activation Functions

The activation function $\sigma$ introduces non-linearity. Let's visualize the common ones:


In [ ]:
def sigmoid(z: np.ndarray) -> np.ndarray:
    """Sigmoid activation function.

    Squashes any input to the range (0, 1).

    Args:
        z: Input array

    Returns:
        Output squeezed between 0 and 1

    Examples:
        >>> sigmoid(np.array([0.0]))
        array([0.5])
        >>> sigmoid(np.array([-100, 100])).round(3)
        array([0., 1.])
        >>> sigmoid(np.array([-1, 0, 1])).round(3)
        array([0.269, 0.5  , 0.731])
    """
    return 1 / (1 + np.exp(-z))


def relu(z: np.ndarray) -> np.ndarray:
    """ReLU (Rectified Linear Unit) activation function.

    Returns max(0, z) for each element.

    Args:
        z: Input array

    Returns:
        Array with negative values set to 0

    Examples:
        >>> relu(np.array([-2, -1, 0, 1, 2]))
        array([0, 0, 0, 1, 2])
        >>> relu(np.array([0.5]))
        array([0.5])
    """
    return np.maximum(0, z)


def tanh_activation(z: np.ndarray) -> np.ndarray:
    """Hyperbolic tangent activation function.

    Squashes input to the range (-1, 1).

    Examples:
        >>> tanh_activation(np.array([0.0]))
        array([0.])
        >>> tanh_activation(np.array([-100, 100]))
        array([-1.,  1.])
    """
    return np.tanh(z)


# Run doctests
# This is how we check our Examples in the docstrings are correct
# We will use this pattern to help with verifying our solutions in the tutorials
doctest.run_docstring_examples(sigmoid, globals(), name="sigmoid")
doctest.run_docstring_examples(relu, globals(), name="relu")
doctest.run_docstring_examples(tanh_activation, globals(), name="tanh_activation")

In [ ]:
# Visualize activation functions
z = np.linspace(-5, 5, 200)
plot_activation_functions_comparison(
    z,
    sigmoid(z),
    relu(z),
    tanh_activation(z),
    same_range=False,
)

### 1.2 Implementing the Perceptron


In [ ]:
class Perceptron:
    """A simple perceptron (artificial neuron).

    Computes: y = activation(w · x + b)

    Attributes:
        weights: Connection weights for each input
        bias: Bias term
        activation: Activation function to apply

    Examples:
        >>> p = Perceptron(n_inputs=2)
        >>> p.weights = np.array([1.0, -1.0])
        >>> p.bias = 0.0
        >>> p.forward(np.array([1.0, 1.0]))  # 1*1 + (-1)*1 + 0 = 0 -> sigmoid(0) = 0.5
        0.5
        >>> p.forward(np.array([2.0, 0.0]))  # 1*2 + (-1)*0 + 0 = 2 -> sigmoid(2) ≈ 0.88
        0.8807970779778823
    """

    def __init__(self, n_inputs: int, activation=sigmoid):
        """Initialize perceptron with random weights.

        Args:
            n_inputs: Number of input connections
            activation: Activation function (default: sigmoid)
        """
        self.n_inputs = n_inputs
        self.activation = activation
        self.init_weights()

    def init_weights(self):
        self.weights = rng.normal(scale=0.5, size=self.n_inputs)
        self.bias = rng.normal(scale=0.1)

    def forward(self, x: np.ndarray) -> float:
        """Compute the perceptron output.

        Args:
            x: Input array of shape (n_inputs,)

        Returns:
            Single output value after activation
        """
        z = np.dot(self.weights, x) + self.bias
        y = self.activation(z)
        return float(y)

In [ ]:
# Test the perceptron
perceptron = Perceptron(n_inputs=3)
test_input = np.array([1.0, 0.5, -0.3])
output = perceptron.forward(test_input)

print("PERCEPTRON")
print("=" * 40)
print(f"Input:   {test_input}")
print(f"Weights: {perceptron.weights.round(3)}")
print(f"Bias:    {perceptron.bias:.3f}")
print(f"Output:  {output:.4f}")

In [ ]:
# Visualize the perceptron computation step-by-step
weighted_inputs = perceptron.weights * test_input
z = np.sum(weighted_inputs) + perceptron.bias
plot_perceptron_computation(weighted_inputs, perceptron.bias, z, output)

### 💭 Let's think about it!

Notice the key properties of the perceptron:

1. **Instantaneous**: Given an input, we immediately get an output. No notion of time.
2. **Stateless**: The same input always produces the same output.
3. **Rate-coded**: The output is a continuous value, interpreted as a "firing rate".
4. **Memoryless**: Previous inputs don't affect current output.


### Exercise 1: Build a Population of Perceptrons

In real neural networks, neurons work together in **layers**. Implement a function that creates multiple perceptrons processing the same input.

**Hints:**

- Each perceptron should have its own random weights
- Return the outputs of all perceptrons as an array


In [ ]:
def perceptron_layer(inputs: np.ndarray, n_neurons: int = 10) -> np.ndarray:
    """Simulate a layer of perceptrons processing the same input.

    Args:
        inputs: Input array of shape (n_features,)
        n_neurons: Number of perceptrons in the layer

    Returns:
        Array of outputs from each perceptron, shape (n_neurons,)

    Examples:
        >>> outputs = perceptron_layer(np.array([1.0, 0.5]), n_neurons=5)
        >>> outputs.shape
        (5,)
        >>> all(0 <= o <= 1 for o in outputs)  # All outputs in sigmoid range
        True
        >>> len(set(outputs.round(4))) > 1  # Different neurons give different outputs
        True
    """
    # TODO: Implement this function!
    # Create n_neurons perceptrons, each processing the input
    # Return array of their outputs
    raise NotImplementedError("Implement perceptron_layer")


doctest.run_docstring_examples(perceptron_layer, globals(), name="perceptron_layer")

In [ ]:
# Visualize the layer response
layer_outputs = perceptron_layer(np.array([0.8, 0.5, 0.9]), n_neurons=20)
plot_layer_response(layer_outputs)

---

## Part 2: The Leaky Integrate-and-Fire (LIF) Neuron

Real neurons are far more complex. They:

- Operate in **continuous time**
- Have **membrane dynamics** (voltage that rises and falls)
- Communicate via discrete **spikes** (action potentials)
- Exhibit **temporal integration** (inputs accumulate over time)

### The LIF Equation

$$\tau_m \frac{dV}{dt} = -(V - V_{rest}) + R \cdot I(t)$$

Where:

- $V$ is the membrane potential (mV)
- $V_{rest}$ is the resting potential (~-70 mV)
- $\tau_m$ is the membrane time constant (how quickly voltage decays)
- $R$ is the membrane resistance
- $I(t)$ is the input current

**Spiking**: When $V \geq V_{threshold}$, the neuron fires and resets to $V_{reset}$.


### Exercise 2: Implement the LIF Voltage Update

The LIF equation describes how the membrane potential changes over time:

$$\tau_m \frac{dV}{dt} = -(V - V_{rest}) + R \cdot I$$

Using **Euler integration**, we can discretize this:

$$dV = \left( -(V - V_{rest}) + R \cdot I \right) \cdot \frac{dt}{\tau_m}$$

$$V_{new} = V + dV$$

**Your task**: Implement a single Euler step of this equation.


In [ ]:
def lif_voltage_step(v: float, v_rest: float, tau_m: float, R: float, I: float, dt: float) -> float:
    """Implement one Euler step of the LIF membrane equation.

    The LIF equation is:
        τ_m * dV/dt = -(V - V_rest) + R * I

    Using Euler integration:
        dV = (-(V - V_rest) + R * I) * (dt / τ_m)
        V_new = V + dV

    Args:
        v: Current membrane potential (mV)
        v_rest: Resting potential (mV)
        tau_m: Membrane time constant (ms)
        R: Membrane resistance (MOhm)
        I: Input current (nA)
        dt: Time step (ms)

    Returns:
        New membrane potential (mV)

    Examples:
        >>> lif_voltage_step(-70, -70, 20, 10, 0, 0.1)  # At rest, no input
        -70.0
        >>> lif_voltage_step(-70, -70, 20, 10, 20, 0.1)  # With input current
        -69.0
        >>> lif_voltage_step(-60, -70, 20, 10, 0, 0.1)  # Above rest, decays back
        -60.05
    """
    # TODO: Implement the Euler step!
    # 1. Calculate dV using the discretized LIF equation
    # 2. Return V + dV
    raise NotImplementedError("Implement lif_voltage_step")


doctest.run_docstring_examples(lif_voltage_step, globals(), name="lif_voltage_step")

In [ ]:
class LIFNeuron:
    """Leaky Integrate-and-Fire neuron model.

    A biologically-inspired neuron that integrates input current
    over time and fires discrete spikes when threshold is reached.

    Attributes:
        tau_m: Membrane time constant (ms)
        v_rest: Resting potential (mV)
        v_reset: Reset potential after spike (mV)
        v_threshold: Firing threshold (mV)
        resistance: Membrane resistance (MOhm)

    Examples:
        >>> lif = LIFNeuron(v_rest=-70, v_threshold=-55)
        >>> lif.v  # Starts at rest
        -70
        >>> lif.step(I=0, dt=1.0)  # No input, no spike
        False
        >>> lif.v  # Still at rest (no input)
        -70.0
    """

    def __init__(
        self,
        tau_m: float = 20.0,
        v_rest: float = -70.0,
        v_reset: float = -80.0,
        v_threshold: float = -55.0,
        resistance: float = 10.0,
    ):
        self.tau_m = tau_m
        self.v_rest = v_rest
        self.v_reset = v_reset
        self.v_threshold = v_threshold
        self.resistance = resistance
        self.v = v_rest

    def reset_state(self):
        """Reset neuron to resting state."""
        self.v = self.v_rest

    def step(self, I: float, dt: float = 0.1) -> bool:
        """Simulate one time step using Euler integration.

        Args:
            I: Input current (nA)
            dt: Time step size (ms)

        Returns:
            True if neuron spiked, False otherwise
        """
        dv = (-(self.v - self.v_rest) + self.resistance * I) * (dt / self.tau_m)
        self.v += dv

        if self.v >= self.v_threshold:
            self.v = self.v_reset
            return True
        return False

    def simulate(self, current: np.ndarray, dt: float = 0.1) -> tuple[np.ndarray, list[float]]:
        """Simulate the neuron over a current time series.

        Args:
            current: Input current array (one value per time step)
            dt: Time step size (ms)

        Returns:
            Tuple of (membrane_potential_trace, spike_times_in_ms)
        """
        self.reset_state()
        v_trace = []
        spike_times = []

        for t_idx, I in enumerate(current):
            spiked = self.step(I, dt)
            v_trace.append(self.v)
            if spiked:
                spike_times.append(t_idx * dt)

        return np.array(v_trace), spike_times

In [ ]:
# Test the LIF neuron with step current
lif = LIFNeuron()
print("LIF NEURON")
print("=" * 40)
print(f"Membrane time constant: {lif.tau_m} ms")
print(f"Resting potential: {lif.v_rest} mV")
print(f"Threshold: {lif.v_threshold} mV")

In [ ]:
# Demonstrate LIF dynamics with step current
dt = 0.1
duration = 200  # ms
time = np.arange(0, duration, dt)

# Step current: off -> on -> off
current = np.zeros_like(time)
current[(time >= 20) & (time < 150)] = 2.0  # 2 nA from 20-150 ms

# Simulate
lif = LIFNeuron()
v_trace, spike_times = lif.simulate(current, dt)

# Plot
plot_lif_response(time, current, v_trace, spike_times, lif.v_threshold, lif.v_rest)

print(f"\nNumber of spikes: {len(spike_times)}")
print(f"Firing rate: {len(spike_times) / (duration / 1000):.1f} Hz")

### 💭 Let's think about it!

Observe the LIF dynamics:

1. **Integration**: The voltage ramps up when current is applied (not instant!)
2. **Leaky**: The voltage decays back to rest when current stops
3. **Spiking**: When threshold is crossed, the neuron fires and resets
4. **Refractory**: After reset, it takes time to reach threshold again

**Try it**: Change the input current magnitude. How does the firing rate change?


### Exercise 2: Implement an F-I Curve

The **F-I curve** (Frequency vs. Input current) is a fundamental characterization of a neuron. Implement a function that computes firing rate for different input currents.

**Hints:**

- Simulate the neuron for each current level
- Compute firing rate = number of spikes / duration (in seconds)


In [ ]:
def compute_fi_curve(currents: np.ndarray, duration: float = 500, dt: float = 0.1) -> np.ndarray:
    """Compute the F-I curve (firing rate vs input current).

    Args:
        currents: Array of input current values to test (nA)
        duration: Simulation duration (ms)
        dt: Time step (ms)

    Returns:
        Array of firing rates (Hz) for each current

    Examples:
        >>> rates = compute_fi_curve(np.array([0.0, 1.0, 2.0, 3.0]))
        >>> rates.shape
        (4,)
        >>> float(rates[0])  # No current = no spikes
        0.0
        >>> bool(rates[1] < rates[2] < rates[3])  # Higher current = higher rate
        True
        >>> all(r >= 0 for r in rates)  # Rates are non-negative
        True
    """
    # TODO: Implement this!
    raise NotImplementedError("Implement compute_fi_curve")


doctest.run_docstring_examples(compute_fi_curve, globals(), name="compute_fi_curve")

In [ ]:
# Compute and plot F-I curve
currents = np.linspace(0, 4, 30)
rates = compute_fi_curve(currents)

# Find rheobase (minimum current for spiking)
rheobase_idx = np.where(rates > 0)[0]
rheobase = currents[rheobase_idx[0]] if len(rheobase_idx) > 0 else None

plot_fi_curve(currents, rates, rheobase)

In [ ]:
# Verify solution with doctests
doctest.run_docstring_examples(compute_fi_curve, globals(), name="compute_fi_curve")

# Compute and plot F-I curve
currents = np.linspace(0, 4, 30)
rates = compute_fi_curve(currents)

# Find rheobase (minimum current for spiking)
rheobase_idx = np.where(rates > 0)[0]
rheobase = currents[rheobase_idx[0]] if len(rheobase_idx) > 0 else None

plot_fi_curve(currents, rates, rheobase)

---

## Part 3: Conductance-Based Synapses

How do neurons receive input? Through **synapses**! When a presynaptic neuron fires, it releases neurotransmitter that opens ion channels in the postsynaptic neuron, creating a **conductance**.

### The Synaptic Conductance Equation

The conductance follows exponential decay after each spike:

$$g_{syn}(t) = \sum_{s} \bar{g}_{max} \cdot e^{-(t-t_s)/\tau} \cdot \Theta(t-t_s)$$

Where:

- $\bar{g}_{max}$ is the maximum conductance per spike (nS)
- $\tau$ is the synaptic time constant (ms)
- $t_s$ is the spike time
- $\Theta$ is the Heaviside step function (0 for negative, 1 for positive)


### Exercise 4: Implement Synaptic Conductance from the Equation

**Your task**: Translate the conductance equation directly into code!

$$g_{syn}(t) = \sum_{s} \bar{g}_{max} \cdot e^{-(t-t_s)/\tau} \cdot \Theta(t-t_s)$$

**Hints:**

- Only include spikes that have already occurred ($t \geq t_s$)
- The Heaviside function $\Theta(x)$ returns 0 for $x < 0$ and 1 for $x \geq 0$
- Use `np.exp()` for the exponential


In [ ]:
def conductance_at_time(t: float, spike_times: list, g_max: float = 1.0, tau: float = 5.0) -> float:
    """Calculate total synaptic conductance at time t from a list of spike times.

    Implements the equation:
        g(t) = Σ g_max * exp(-(t - t_s) / τ) * Θ(t - t_s)

    Where Θ is the Heaviside step function (0 for x<0, 1 for x≥0).

    Args:
        t: Current time (ms)
        spike_times: List of presynaptic spike times (ms)
        g_max: Maximum conductance per spike (nS)
        tau: Decay time constant (ms)

    Returns:
        Total conductance at time t (nS)

    Examples:
        >>> conductance_at_time(10, [10], g_max=2.0, tau=5.0)  # At spike time
        2.0
        >>> round(conductance_at_time(15, [10], g_max=2.0, tau=5.0), 3)  # After 1 tau
        0.736
        >>> conductance_at_time(10, [20], g_max=2.0, tau=5.0)  # Before spike
        0.0
        >>> round(conductance_at_time(15, [10, 12], g_max=2.0, tau=5.0), 3)  # Two spikes sum
        1.645
    """
    # TODO: Implement this!
    # 1. Loop through each spike time
    # 2. Only include spikes that have occurred
    # 3. Sum up g for each valid spike
    raise NotImplementedError("Implement conductance_at_time")


doctest.run_docstring_examples(conductance_at_time, globals(), name="conductance_at_time")

In [ ]:
class Synapse:
    """Conductance-based synapse model.

    Models synaptic transmission with exponential conductance decay.

    Attributes:
        g_max: Maximum conductance (nS)
        tau_syn: Synaptic time constant (ms)
        e_syn: Reversal potential (mV)

    Examples:
        >>> syn = Synapse(g_max=2.0, tau_syn=5.0)
        >>> syn.g  # Initially no conductance
        0.0
        >>> syn.receive_spike()
        >>> syn.g  # After spike: g_max
        2.0
        >>> syn.step(dt=5.0)  # After one time constant
        >>> round(syn.g, 3)  # Decayed to g_max * exp(-1)
        0.736
    """

    def __init__(self, g_max: float = 1.0, tau_syn: float = 5.0, e_syn: float = 0.0):
        self.g_max = g_max
        self.tau_syn = tau_syn
        self.e_syn = e_syn
        self.g = 0.0

    def reset_state(self):
        """Reset synapse to baseline."""
        self.g = 0.0

    def receive_spike(self):
        """Process an incoming presynaptic spike."""
        self.g += self.g_max

    def step(self, dt: float = 0.1) -> None:
        """Update conductance (exponential decay)."""
        self.g *= np.exp(-dt / self.tau_syn)

    def get_current(self, v_post: float) -> float:
        """Calculate synaptic current given postsynaptic voltage.

        Args:
            v_post: Postsynaptic membrane potential (mV)

        Returns:
            Synaptic current (nA)
        """
        return self.g * (self.e_syn - v_post)

In [ ]:
# Demonstrate synaptic conductance with multiple spikes
dt = 0.1
time = np.arange(0, 150, dt)
spike_times_pre = [10, 30, 35, 40, 80, 120]  # Note the burst at 30-40 ms!

synapse = Synapse(g_max=2.0, tau_syn=5.0)
g_trace = []
i_trace = []  # Track current too

v_post = -70  # Assume constant postsynaptic voltage for now

for t in time:
    if any(abs(t - st) < dt / 2 for st in spike_times_pre):
        synapse.receive_spike()
    g_trace.append(synapse.g)
    i_trace.append(synapse.get_current(v_post))
    synapse.step(dt)

# Plot
plot_synaptic_response(time, g_trace, i_trace, spike_times_pre)

### 💭 Let's think about it!

Notice the **temporal summation** at 30-40 ms! When spikes arrive close together:

- The conductance **adds up** before it can decay
- The response is **larger** than for isolated spikes
- This is a form of **temporal integration** the perceptron cannot capture

**Exploration**: What happens if you change `tau_syn`? Try 2 ms vs 20 ms.


---

## Part 4: Short-Term Plasticity (Vesicle Depletion)

Real synapses don't have unlimited neurotransmitter! Each spike releases vesicles from a **finite pool**, and the pool takes time to replenish.

### The Tsodyks-Markram Model (Simplified)

We track:

- $x$: Fraction of available vesicles (0 to 1)
- $u$: Release probability per spike

On each spike:

- Vesicles used: $\Delta x = u \cdot x$
- Effective conductance: $g_{eff} = \bar{g} \cdot u \cdot x$

Between spikes, vesicles recover:
$$\frac{dx}{dt} = \frac{1 - x}{\tau_{rec}}$$

This creates **synaptic depression**: repeated firing depletes vesicles!


In [ ]:
class DepressingSynapse:
    """Synapse with short-term depression (vesicle depletion).

    Implements a simplified Tsodyks-Markram model where repeated
    activation depletes the vesicle pool, reducing synaptic strength.

    Attributes:
        g_max: Maximum conductance when fully recovered (nS)
        tau_syn: Conductance decay time constant (ms)
        tau_rec: Vesicle recovery time constant (ms)
        u: Release probability per spike
        e_syn: Reversal potential (mV)

    Examples:
        >>> syn = DepressingSynapse(g_max=4.0, u=0.5)
        >>> syn.x  # Full vesicle pool
        1.0
        >>> syn.receive_spike()
        >>> syn.g  # First spike: g_max * u * x = 4 * 0.5 * 1 = 2
        2.0
        >>> syn.x  # Pool depleted: 1 - 0.5*1 = 0.5
        0.5
        >>> syn.receive_spike()  # Second spike immediately
        >>> syn.g  # Weaker: previous g + 4 * 0.5 * 0.5 = 2 + 1 = 3
        3.0
    """

    def __init__(
        self,
        g_max: float = 1.0,
        tau_syn: float = 5.0,
        tau_rec: float = 200.0,
        u: float = 0.5,
        e_syn: float = 0.0,
    ):
        self.g_max = g_max
        self.tau_syn = tau_syn
        self.tau_rec = tau_rec
        self.u = u
        self.e_syn = e_syn
        self.g = 0.0
        self.x = 1.0  # Full pool

    def reset_state(self):
        """Reset to fully recovered state."""
        self.g = 0.0
        self.x = 1.0

    def receive_spike(self):
        """Process incoming spike with vesicle depletion."""
        released = self.u * self.x
        self.x -= released
        self.g += self.g_max * released

    def step(self, dt: float = 0.1) -> None:
        """Update conductance decay and vesicle recovery."""
        self.g *= np.exp(-dt / self.tau_syn)
        self.x += (1 - self.x) * (dt / self.tau_rec)

    def get_current(self, v_post: float) -> float:
        """Calculate synaptic current."""
        return self.g * (self.e_syn - v_post)

In [ ]:
# Compare regular vs depressing synapse
dt = 0.1
time = np.arange(0, 500, dt)

# Regular spike train at 20 Hz
spike_times_pre = np.arange(50, 400, 50)

# Regular synapse
syn_regular = Synapse(g_max=2.0, tau_syn=5.0)
g_regular = []

# Depressing synapse
syn_depressing = DepressingSynapse(g_max=4.0, tau_syn=5.0, tau_rec=200.0, u=0.5)
g_depressing = []
x_trace = []

for t in time:
    is_spike = any(abs(t - st) < dt / 2 for st in spike_times_pre)

    if is_spike:
        syn_regular.receive_spike()
        syn_depressing.receive_spike()

    g_regular.append(syn_regular.g)
    g_depressing.append(syn_depressing.g)
    x_trace.append(syn_depressing.x)

    syn_regular.step(dt)
    syn_depressing.step(dt)

# Plot comparison
plot_synapse_comparison(
    time, np.array(g_regular), np.array(g_depressing), np.array(x_trace), spike_times_pre
)

### 💭 Let's think about it!

Compare the two synapse types:

1. **Regular synapse**: Each spike → same response. **Stateless**.
2. **Depressing synapse**: First spike → strong. Later spikes → weaker. Has **memory**!

**Why is this computationally useful?**

- **High-pass filter**: Responds strongly to _onset_ of activity
- **Gain control**: Prevents runaway excitation
- **Temporal differentiation**: Sensitive to _changes_ in input rate


### Exercise 3: Explore STP Parameters

Investigate how different short-term plasticity parameters affect synaptic transmission.

**Try:**

1. Fast recovery (`tau_rec=50`): Does depression still occur?
2. High release probability (`u=0.8`): How does the first vs later responses compare?
3. Low release probability (`u=0.2`): What changes?


In [ ]:
# Your exploration here!
# Create synapses with different parameters and compare their responses

# Example setup:
params = [
    {"name": "Standard", "tau_rec": 200, "u": 0.5, "color": "blue"},
    {"name": "Fast Recovery", "tau_rec": 50, "u": 0.5, "color": "green"},
    {"name": "High Release", "tau_rec": 200, "u": 0.8, "color": "red"},
]

# TODO: For each parameter set:
# 1. Create a DepressingSynapse with those parameters
# 2. Simulate responses to a spike train
# 3. Plot and compare the results
plot_stp_comparison(...)

### Exercise 5: Implement a Facilitating Synapse

So far we've seen **depressing** synapses that get weaker with repeated use. But some synapses show the opposite behavior: **short-term facilitation** where the release probability _increases_ after each spike!

**The Mechanism:**

- Each spike temporarily increases the release probability $u$
- $u$ decays back to baseline $u_{base}$ over time
- This creates responses that get _stronger_ with repeated activation

**Your task**: Implement a `FacilitatingSynapse` class with the following dynamics:

- On each spike: $u \leftarrow u + U_{step} \cdot (u_{max} - u)$, then release conductance $g_{max} \cdot u$
- Between spikes: $u$ decays toward $u_{base}$ with time constant $\tau_{facil}$


In [ ]:
class FacilitatingSynapse:
    """Synapse with short-term facilitation.

    Unlike depressing synapses, facilitating synapses get STRONGER
    with repeated use. The release probability u increases after
    each spike and decays back to baseline between spikes.

    Attributes:
        g_max: Maximum conductance (nS)
        tau_syn: Conductance decay time constant (ms)
        tau_facil: Facilitation decay time constant (ms)
        u_base: Baseline release probability
        u_max: Maximum release probability
        u_step: Facilitation increment per spike
        e_syn: Reversal potential (mV)

    Examples:
        >>> syn = FacilitatingSynapse(g_max=4.0, u_base=0.1, u_max=0.8, u_step=0.3)
        >>> syn.u  # Starts at baseline
        0.1
        >>> syn.receive_spike()
        >>> round(syn.g, 2)  # First spike: weak (g_max * 0.1 = 0.4)
        0.4
        >>> round(syn.u, 2)  # u increased toward u_max
        0.31
        >>> syn.receive_spike()  # Second spike immediately
        >>> round(syn.g, 2)  # Stronger! (0.4 + 4 * 0.31 = 1.64)
        1.64
    """

    def __init__(
        self,
        g_max: float = 1.0,
        tau_syn: float = 5.0,
        tau_facil: float = 100.0,
        u_base: float = 0.1,
        u_max: float = 0.8,
        u_step: float = 0.3,
        e_syn: float = 0.0,
    ):
        # TODO: Initialize all attributes
        # - g_max, tau_syn, tau_facil, u_base, u_max, u_step, e_syn
        # - g: current conductance (starts at 0)
        # - u: current release probability (starts at u_base)
        raise NotImplementedError("Implement __init__")

    def reset_state(self):
        """Reset to baseline state."""
        # TODO: Reset g to 0 and u to u_base
        raise NotImplementedError("Implement reset_state")

    def receive_spike(self):
        """Process incoming spike with facilitation.

        1. Add conductance
        2. Increase u toward u_max
        """
        # TODO: Implement facilitation dynamics
        raise NotImplementedError("Implement receive_spike")

    def step(self, dt: float = 0.1) -> None:
        """Update conductance decay and u decay back to baseline.

        - g decays exponentially with tau_syn
        - u decays toward u_base with tau_facil
        """
        # TODO: Implement decay dynamics
        raise NotImplementedError("Implement step")

    def get_current(self, v_post: float) -> float:
        """Calculate synaptic current."""
        return self.g * (self.e_syn - v_post)


doctest.run_docstring_examples(FacilitatingSynapse.__doc__, globals(), name="FacilitatingSynapse")

In [ ]:
# Compare depressing vs facilitating synapses
import matplotlib.pyplot as plt

dt = 0.1
time = np.arange(0, 500, dt)
spike_times_pre = np.arange(50, 400, 50)

# Depressing synapse
syn_dep = DepressingSynapse(g_max=4.0, tau_syn=5.0, tau_rec=200.0, u=0.5)
g_dep = []

# Facilitating synapse
syn_fac = FacilitatingSynapse(
    g_max=4.0, tau_syn=5.0, tau_facil=100.0, u_base=0.1, u_max=0.8, u_step=0.3
)
g_fac = []
u_trace = []

for t in time:
    is_spike = any(abs(t - st) < dt / 2 for st in spike_times_pre)

    if is_spike:
        syn_dep.receive_spike()
        syn_fac.receive_spike()

    g_dep.append(syn_dep.g)
    g_fac.append(syn_fac.g)
    u_trace.append(syn_fac.u)

    syn_dep.step(dt)
    syn_fac.step(dt)

# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

axes[0].plot(time, g_dep, "b-", label="Depressing", linewidth=1.5)
axes[0].plot(time, g_fac, "r-", label="Facilitating", linewidth=1.5)
for st in spike_times_pre:
    axes[0].axvline(st, color="gray", alpha=0.3, linestyle="--")
axes[0].set_ylabel("Conductance (nS)")
axes[0].legend()
axes[0].set_title("Depression vs Facilitation: Opposite Temporal Filters")

axes[1].plot(time, u_trace, "r-", label="u (facilitating)", linewidth=1.5)
axes[1].axhline(syn_fac.u_base, color="gray", linestyle=":", label=f"u_base={syn_fac.u_base}")
for st in spike_times_pre:
    axes[1].axvline(st, color="gray", alpha=0.3, linestyle="--")
axes[1].set_xlabel("Time (ms)")
axes[1].set_ylabel("Release probability u")
axes[1].legend()

plt.tight_layout()
plt.show()

print("Depressing synapse: First spike is STRONGEST (high-pass filter)")
print("Facilitating synapse: Later spikes are STRONGER (low-pass filter)")

---

## Part 5: Putting It All Together

Now let's connect a depressing synapse to an LIF neuron - the complete biological neuron model!


In [ ]:
class LIFWithSynapse:
    """LIF neuron with conductance-based synaptic input.

    Combines LIF membrane dynamics with a depressing synapse.

    Examples:
        >>> neuron = LIFWithSynapse(g_max=5.0)
        >>> neuron.v == neuron.v_rest
        True
        >>> neuron.synapse.x  # Full vesicle pool
        1.0
    """

    def __init__(
        self,
        tau_m: float = 20.0,
        v_rest: float = -70.0,
        v_reset: float = -80.0,
        v_threshold: float = -55.0,
        g_max: float = 5.0,
        tau_syn: float = 5.0,
        tau_rec: float = 200.0,
        u: float = 0.5,
        e_syn: float = 0.0,
    ):
        self.tau_m = tau_m
        self.v_rest = v_rest
        self.v_reset = v_reset
        self.v_threshold = v_threshold
        self.e_syn = e_syn
        self.synapse = DepressingSynapse(g_max, tau_syn, tau_rec, u, e_syn)
        self.v = v_rest

    def reset_state(self):
        self.v = self.v_rest
        self.synapse.reset_state()

    def step(self, presynaptic_spike: bool, dt: float = 0.1) -> bool:
        """Simulate one time step."""
        if presynaptic_spike:
            self.synapse.receive_spike()

        I_syn = self.synapse.get_current(self.v)
        dv = (-(self.v - self.v_rest) + I_syn) * (dt / self.tau_m)
        self.v += dv
        self.synapse.step(dt)

        if self.v >= self.v_threshold:
            self.v = self.v_reset
            return True
        return False

    def simulate(self, presynaptic_spikes: np.ndarray, dt: float = 0.1):
        """Simulate given presynaptic spike train."""
        self.reset_state()
        v_trace, g_trace, x_trace = [], [], []
        spike_times = []

        for t_idx, is_spike in enumerate(presynaptic_spikes):
            spiked = self.step(is_spike, dt)
            v_trace.append(self.v)
            g_trace.append(self.synapse.g)
            x_trace.append(self.synapse.x)
            if spiked:
                spike_times.append(t_idx * dt)

        return np.array(v_trace), np.array(g_trace), np.array(x_trace), spike_times

In [ ]:
# Simulate with Poisson input
dt = 0.1
duration = 1000
time = np.arange(0, duration, dt)

# Poisson spike train (~30 Hz)
rate = 30
p_spike = rate * dt / 1000
presynaptic_spikes = rng.random(len(time)) < p_spike

# Simulate
neuron = LIFWithSynapse(g_max=8.0, tau_rec=150.0, u=0.4)
v_trace, g_trace, x_trace, spike_times = neuron.simulate(presynaptic_spikes, dt)

print(
    f"Presynaptic spikes: {presynaptic_spikes.sum()} ({presynaptic_spikes.sum() / (duration / 1000):.1f} Hz)"
)
print(f"Postsynaptic spikes: {len(spike_times)} ({len(spike_times) / (duration / 1000):.1f} Hz)")

In [ ]:
# Comprehensive visualization
plot_complete_neuron(
    time, presynaptic_spikes, g_trace, x_trace, v_trace, spike_times, neuron.v_threshold
)

---

## Part 6: Side-by-Side Comparison

Let's compare how the perceptron and biological neuron respond to varying input intensities.


In [ ]:
def compare_io_curves(intensities: np.ndarray, duration: float = 500, dt: float = 0.1):
    """Compare I-O curves for perceptron and LIF."""
    # Perceptron
    p = Perceptron(n_inputs=1)
    p.weights = np.array([1.0])
    p.bias = 0.0
    p_outputs = [p.forward(np.array([i])) for i in intensities]

    # LIF with synapse
    lif = LIFWithSynapse(g_max=10.0, tau_rec=150.0, u=0.4)
    lif_rates = []

    for intensity in intensities:
        rate = max(0, intensity * 50)
        time = np.arange(0, duration, dt)
        spikes = rng.random(len(time)) < (rate * dt / 1000)
        _, _, _, spike_times = lif.simulate(spikes, dt)
        lif_rates.append(len(spike_times) / (duration / 1000))

    return np.array(p_outputs), np.array(lif_rates)


intensities = np.linspace(0, 3, 25)
p_outputs, lif_rates = compare_io_curves(intensities)

# Plot comparison
plot_io_comparison(intensities, p_outputs, intensities * 50, lif_rates)

---

## Summary

In this tutorial, you implemented:

1. **The Perceptron** - Instantaneous, stateless weighted sum + activation
2. **The LIF Neuron** - Dynamic membrane potential with temporal integration
3. **Conductance-Based Synapses** - Realistic input: spikes → conductance → current
4. **Short-Term Plasticity** - Vesicle depletion creates history-dependent transmission

### Key Differences

| Aspect         | Perceptron           | Biological Neuron           |
| -------------- | -------------------- | --------------------------- |
| **Input**      | Static vector        | Spike train → conductance   |
| **Processing** | Instant weighted sum | Temporal integration + leak |
| **Output**     | Continuous value     | Discrete spike train        |
| **Memory**     | None                 | Membrane state, vesicles    |
| **Plasticity** | Fixed weights        | Short-term depression       |

### Key Takeaways

- **Time matters**: Biological neurons process information over time
- **History matters**: Short-term plasticity = context-dependent processing
- **Spikes are sparse**: Discrete events, not continuous rates
- **Neither is "better"**: Each captures different computational principles

### References

1. Gerstner, W., et al. (2014). _Neuronal Dynamics_. Cambridge University Press.
2. Tsodyks, M., & Markram, H. (1997). The neural code between neocortical pyramidal neurons depends on neurotransmitter release probability. _PNAS_, 94(2), 719-723.
3. Abbott, L. F., & Regehr, W. G. (2004). Synaptic computation. _Nature_, 431(7010), 796-803.
